# DRIN Preprocessing Script 3 - Year Split and Specified Annual demand profile calculations

This notebook is intended to calculate the Year Split and Specified Annual demand profile based on the load curve data for Albania. 

Note: 

* Year Split = (total number of hours in each time slice)/ total number of hours in a year

* Specified demand profile = (sum.hours in each time slice * sum.of demanded energy in each time slice)/Total annual energy


### 1. Import packages and dependencies. 


In [ ]:
%load_ext autoreload
import sys
sys.path.append("..") #this is to add the above folder to the package directory
import os
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# from pandas.plotting import register_matplotlib_converters
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline 

### 2. Read the input file and setup the dataframe 

In [19]:
folder_path = os.path.join('..','data')

load_curve = os.path.join(folder_path, 'Albania_load_curve.xlsx')

load = pd.read_excel(load_curve) #Note that the unit of load in the data file is MW.

In [40]:
load['year'] = load['Date'].dt.year 
load['month'] = load['Date'].dt.month
load['week'] = load['Date'].dt.week
load['day'] = load['Date'].dt.day 
load['hours'] = (1+load['Date'].dt.hour)/(1+load['Date'].dt.hour) 
load.rename(columns={'Load':'Power(MW)'}, inplace=True)


<ipython-input-40-7a439694c7db>:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  load['week'] = load['Date'].dt.week


### 3. Calculating the demand profile and yearsplit: 

In [41]:
# Demand profile:
demand_profile = pd.pivot_table(load, values=['Power(MW)','hours'], index='week', aggfunc=np.sum, fill_value=0)
demand_profile.head()

# Calculating the year split and specified demand profile:
demand_profile2 = load.groupby('week')['Power(MW)','hours'].sum()
demand_profile2['year'] = load.groupby('week')['year'].mean()
demand_profile2['yearsplit'] = (8760/52)/8760
demand_profile2['TSdemand'] = (demand_profile2['Power(MW)']*demand_profile2['hours'])
demand_profile2['specifieddemandprofile'] = demand_profile2['TSdemand']/(demand_profile2['TSdemand'].sum())
demand_profile2.reset_index(inplace=True)


# Creating a dataframe for each parameter
df_sdp = demand_profile2[['week','specifieddemandprofile']].copy()
df_ys = demand_profile2[['week','yearsplit']].copy()



<ipython-input-41-3eb8ce314e8b>:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  demand_profile2 = load.groupby('week')['Power(MW)','hours'].sum()


### 4 Saving the CF tables for each technology and country

In [39]:
output_folder = os.path.join('processed_data', 'demand_profile')
os.makedirs(output_folder, exist_ok = True)

file1_path=os.path.join(output_folder, 'Specified_demand_profile.xlsx')
file2_path=os.path.join(output_folder, 'year_split.xlsx')


for i in range(20,56):
    df_sdp['20{}'.format(i)]=df_sdp['specifieddemandprofile'].round(3)
    #df_sdp.drop('specifieddemandprofile',axis=1, inplace=True)
    df_sdp.rename({'2055':'2055:=','week':' '},axis=1, inplace=True)
    
    df_ys['20{}'.format(i)]=df_ys['yearsplit'].round(3)
    #df_ys.drop('yearsplit',axis=1, inplace=True)
    df_ys.rename({'2055':'2055:=','week':' '},axis=1, inplace=True)
    

df_sdp.drop('specifieddemandprofile',axis=1, inplace=True)
df_ys.drop('yearsplit',axis=1, inplace=True)
writer1 = pd.ExcelWriter(file1_path, engine='xlsxwriter')
writer2 = pd.ExcelWriter(file2_path, engine='xlsxwriter')
df_sdp.to_excel(writer1, index=False)
df_ys.to_excel(writer2, index=False)

writer1.save()
writer2.save()

# End